# Day 9: RAG Systems - Mastering Document Intelligence

## Going Beyond Basic File Reading

Yesterday you used the `files` parameter for automatic RAG. Today you'll learn how RAG actually works!

### Today's Learning:
1. RAG workflow deep dive
2. Chunking strategies
3. Embedding and retrieval
4. ParallelDocQA agent
5. Performance optimization
6. When to use what

Let's master document intelligence! 📚

In [ ]:
import os
os.environ['FIREWORKS_API_KEY'] = 'fw_3ZTLPrnEtuscTUPYy3sYx3ag'
llm_cfg = {
    'model': 'accounts/fireworks/models/qwen3-235b-a22b-thinking-2507',
    'model_server': 'https://api.fireworks.ai/inference/v1',
    'api_key': os.environ['FIREWORKS_API_KEY'],
    'generate_cfg': {'max_tokens': 32768}
}
print('✅ Configured')

## Part 1: How RAG Works

RAG has 5 steps:
1. **Ingestion**: Read documents
2. **Chunking**: Split into pieces
3. **Embedding**: Convert to vectors
4. **Retrieval**: Find relevant chunks
5. **Generation**: LLM uses chunks to answer

## Part 2: Assistant with RAG (Review)

In [ ]:
from qwen_agent.agents import Assistant

# Create document
with open('ai_guide.txt', 'w') as f:
    f.write('AI Guide: LLMs are trained on vast text. RAG adds specific knowledge.')

bot = Assistant(llm=llm_cfg, files=[os.path.abspath('ai_guide.txt')])
messages = [{'role': 'user', 'content': 'What does RAG add?'}]

for response in bot.run(messages):
    print(response[-1].get('content', ''))

## Part 3: ParallelDocQA Agent

For VERY long documents or multiple documents, use ParallelDocQA:

In [ ]:
# ParallelDocQA example (from parallel_doc_qa.py)
# from qwen_agent.agents.doc_qa import ParallelDocQA

# parallel_bot = ParallelDocQA(
#     llm={'model': 'qwen-plus-latest', 'generate_cfg': {'max_retries': 10}}
# )

# More efficient for 100+ page documents
print('ParallelDocQA: Advanced RAG for long documents')

## Summary

✅ **RAG workflow** - 5 steps explained
✅ **Assistant files** - Easy RAG
✅ **ParallelDocQA** - For long docs
✅ **Chunking** - Size matters
✅ **Performance** - Optimization tips

**Tomorrow**: Multi-Agent Systems! 🤖🤖